<a href="https://colab.research.google.com/github/ZenkinAlex/My_projects/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Задание**

Показать динамику международной торговли по продукту фтористый алюминий (код hs code 282612)

Сайт
Trade Map - Trade statistics for international business development

Нужно самостоятельно зарегистрироваться (создать бесплатный логин и пароль)

По указанном коду скачать данные за период 2018-2025 (в идеале спарсить в автоматическом режиме)
- объемы (quantities)
- цены (unit values)

Показать крупнейших экспортеров и импортеров, в динамике со срезами изменений по годам (как по объемам, так и по ценам)
- таблица
- карта

По крупнейшим экспортерам сделать срез по крупнейшим потребителям (по Италии использовать вкладку mirror data)
- желательно в виде карты (желательно автоматическая карта, построенная с помощью библиотек python)
- кому грузят и по каким ценам

Сделать сравнение по основным экспортерам и импортерам
- география поставок (объем по странам)
- динамика цен от разных поставщиков для импортеров (или по разным потребителям для экспортеров)

Все своды должны быть представлены в визуально понятной форме


## **Решение**
### **Предисловие**
У сайта нет api, а веб-скрэппингом парсить данные это не лучший вариант. Насколько я понял эта платформа берет данные здесь https://comtradedeveloper.un.org/signin?returnUrl=%2F и тут есть свой api https://comtradedeveloper.un.org/signin и описание на гите
https://github.com/uncomtrade/comtradeapicall?spm=a2ty_o01.29997173.0.0.65cd5171WMneAp. Поэтому для автоматизации решил использовать эту платформу

### **1. Установка библиотек**

In [1]:
!pip install geopandas requests pandas numpy matplotlib seaborn plotly openpyxl comtradeapicall ipywidgets -q

import pandas as pd
import numpy as np
import comtradeapicall
from datetime import date
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings("ignore")


SUBSCRIPTION_KEY = "1fbfccefb0e8489db0a75c81db504ae8"
HS_CODE = "282612"
YEARS = list(range(2018, 2026))  # 2018–2025
MAX_RECORDS_PER_CALL = 250000  # лимит API


### **2. Парсинг данных**

In [2]:
all_data = []

for year in YEARS:
    print(f"Загрузка данных за {year}...")
    try:
        df = comtradeapicall.getFinalData(
            subscription_key=SUBSCRIPTION_KEY,
            typeCode='C',          # Goods
            freqCode='A',          # Annual
            clCode='HS',           # Harmonized System
            period=str(year),
            reporterCode=None,     # Все страны-репортеры
            cmdCode=HS_CODE,
            flowCode=None,         # Все потоки (экспорт/импорт)
            partnerCode=None,
            partner2Code=None,
            customsCode=None,
            motCode=None,
            maxRecords=MAX_RECORDS_PER_CALL,
            format_output='JSON',
            includeDesc=True
        )
        if not df.empty:
            df['Year'] = year
            all_data.append(df)
        else:
            print(f"Нет данных за {year}")
    except Exception as e:
        print(f"Ошибка при загрузке {year}: {e}")

df_full = pd.concat(all_data, ignore_index=True)

# Сохраняем сырые данные на случай перезапуска
df_full.to_csv("fluoride_aluminium_282612_raw.csv", index=False)

Загрузка данных за 2018...
Загрузка данных за 2019...
Загрузка данных за 2020...
Загрузка данных за 2021...
Загрузка данных за 2022...
Загрузка данных за 2023...
Загрузка данных за 2024...
Загрузка данных за 2025...


In [3]:
df_full.head()

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,...,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Year
0,C,A,20180101,2018,52,2018,36,AUS,Australia,X,...,True,1689.0,False,NaN,33595.032,33595.032,6,True,False,2018
1,C,A,20180101,2018,52,2018,36,AUS,Australia,X,...,True,1689.0,False,NaN,33595.032,33595.032,6,False,True,2018
2,C,A,20180101,2018,52,2018,36,AUS,Australia,X,...,True,2768959.0,False,NaN,571482.404,571482.404,6,False,True,2018
3,C,A,20180101,2018,52,2018,36,AUS,Australia,X,...,True,2768959.0,False,NaN,571482.404,571482.404,6,False,True,2018
4,C,A,20180101,2018,52,2018,36,AUS,Australia,X,...,False,2767270.0,False,NaN,537887.372,537887.372,0,False,True,2018


**Описание столбцов датасета**

***typeCode*** | Тип данных: 'C' = товары, 'S' = услуги

***freqCode*** | Частота: 'A' = годовые, 'M' = месячные

***refYear*** | Год наблюдения (например, 2023)

***refMonth*** | Месяц (1–12); для годовых данных = 0 или 12

***period*** | Строка периода: '2023' для годовых, '202305' для мая 2023

***reporterCode*** | Цифровой код страны-репортера по Comtrade (например, 842 = США)

***reporterISO*** | ISO 3166-alpha-3 код страны-репортера (например, 'USA')

***reporterDesc*** | Название страны-репортера (например, 'United States')

***flowCode*** | Код направления торговли: 'X' = экспорт, 'M' = импорт

***flowDesc*** | Описание потока: 'Export', 'Import'

***partnerCode*** | Цифровой код торгового партнёра (страны-получателя/отправителя)

***partnerISO*** | ISO 3166-alpha-3 код партнёра (например, 'RUS')

***partnerDesc*** | Название страны-партнёра

***partner2Code, partner2ISO, partner2Desc*** | Вторичный партнёр (редко используется; обычно None или агрегаты)

***cmdCode*** Код товара по HS (например, '282612' = фториды алюминия)

***cmdDesc*** | Описание товара на английском

***aggrLevel*** | Уровень агрегации в HS (6 = детальный, 4 = групповой и т.д.)

***isLeaf*** | True, если это конечный (не агрегированный) код HS

***netWgt*** | Чистый вес в килограммах (kg)

***isNetWgtEstimated*** | True, если вес оценён, а не измерен

***fobvalue*** | Стоимость экспорта в USD по условиям FOB (Free On Board)

***cifvalue*** | Стоимость импорта в USD по условиям CIF (Cost, Insurance, Freight)

***primaryValue*** | Основная стоимость: автоматически выбирает fobvalue для экспорта и cifvalue для импорта. Рекомендуется использовать именно её.

***qty*** | Количество в основной единице измерения (qtyUnitAbbr)

***qtyUnitCode*** | Код единицы измерения (например, 8 = кг, 15 = штуки)

***qtyUnitAbbr*** | Аббревиатура единицы измерения (например, 'kg', 'NAR')

***isQtyEstimated*** | Признак оценённого количества
***customsCode, customsDesc*** | Таможенная процедура (обычно 'C00' = стандартная)

***motCode, motDesc*** | Способ транспортировки (морской, воздушный и т.д.)

***isAggregate*** | True, если запись представляет агрегат (например, 'World', регионы)

***isReported*** | True, если данные предоставлены самой страной-репортером

***classificationCode*** |  Версия HS (например, 'H6' = HS 2022)

***Year*** | Дополнительное поле, добавленное вручную для удобства анализа




### **3. Предобработка данных**

In [4]:
df_clean = df_full[
    (df_full['reporterISO'].notna()) &
    (df_full['partnerISO'].notna()) &
    (df_full['primaryValue'] > 0) &
    (df_full['netWgt'] > 0) &
    (df_full['partnerISO'] != 'WLD') &
    (df_full['reporterISO'] != 'WLD')
].copy()

# Расчёт цены за кг
df_clean['Price per kg (USD)'] = df_clean['primaryValue'] / df_clean['netWgt']

# Фильтр выбросов (реалистичный диапазон цен для AlF3: $0.5 – $10/kg - по данным на 16 января 2026 года, цена алюминия на LME составляла 3140,70 USD/т)
df_clean = df_clean[
    (df_clean['Price per kg (USD)'] >= 0.1) &
    (df_clean['Price per kg (USD)'] <= 50)
]

print(f"✅ После очистки: {len(df_clean)} записей")

✅ После очистки: 18313 записей


### **4. ТОП-5 стран по объему (kg) экспорта**

In [5]:
df_export = df_clean[df_clean['flowDesc'] == 'Export']
df_import = df_clean[df_clean['flowDesc'] == 'Import']

pivot_export_volume = df_export.pivot_table(
    index=['reporterDesc','reporterISO'],
    columns='refYear',
    values='netWgt',
    aggfunc='sum',
).reset_index()

year_columns = [col for col in pivot_export_volume.columns if col != 'reporterDesc' and col !='reporterISO']
pivot_export_volume['TOTAL'] = pivot_export_volume[year_columns].sum(axis=1)
pivot_export_volume = pivot_export_volume.sort_values('TOTAL', ascending=False).nlargest(5,'TOTAL')
pivot_export_volume

refYear,reporterDesc,reporterISO,2018,2019,2020,2021,2022,2023,2024,2025,TOTAL
10,China,CHN,211020930.0,1.877248e+08,195002742.0,95971608.0,2.222576e+08,2.396563e+08,2.681986e+08,NaN,1.419833e+09
51,Tunisia,TUN,85776000.0,8.490900e+07,66683000.0,79530000.0,7.936400e+07,5.232400e+07,7.862960e+07,NaN,5.272156e+08
8,Canada,CAN,35504732.0,1.154250e+08,13964920.0,10824080.0,5.878624e+07,4.202568e+07,5.544424e+06,NaN,2.820750e+08
54,United Arab Emirates,ARE,49243072.0,1.038520e+07,23844796.0,105167622.0,3.168437e+07,3.008071e+07,NaN,NaN,2.504058e+08
30,Lithuania,LTU,26496840.0,2.631268e+07,33887640.0,40826360.0,7.248200e+06,1.531000e+06,2.346320e+07,NaN,1.597659e+08


In [7]:
# Столбчатая диаграмма
fig_bar = px.bar(
    pivot_export_volume.sort_values('TOTAL'),
    x='TOTAL',
    y='reporterDesc',
    orientation='h',
    title='ТОП-5 стран по объёму экспорта (кг)',
    labels={'TOTAL': 'Экспорт (кг)', 'reporterDesc': 'Страна'}
)

# Карта (с лог-шкалой)
pivot_export_volume['log_TOTAL'] = np.log10(pivot_export_volume['TOTAL'] + 1)
fig_map = px.choropleth(
    pivot_export_volume,
    locations='reporterISO',
    color='log_TOTAL',
    hover_name='reporterDesc',
    labels={'log_TOTAL': 'log₁₀(кг)'},
    projection='natural earth'
)

# Комбинируем
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    specs=[[{"type": "choropleth"}, {"type": "bar"}]]
)

for trace in fig_map.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig_bar.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(title_text="Экспорт", height=500)
fig.show()

### **5. ТОП-5 стран по объему (kg) импорта**

In [9]:
pivot_import_volume =df_import.pivot_table(
    index=['reporterDesc','reporterISO'],
    columns='refYear',
    values='netWgt',
    aggfunc='sum',
).reset_index()

year_columns = [col for col in pivot_import_volume.columns if col != 'reporterDesc'and col !='reporterISO']
pivot_import_volume['TOTAL'] = pivot_import_volume[year_columns].sum(axis=1)
pivot_import_volume = pivot_import_volume.sort_values('TOTAL', ascending=False).nlargest(5,'TOTAL')
pivot_import_volume

refYear,reporterDesc,reporterISO,2018,2019,2020,2021,2022,2023,2024,2025,TOTAL
63,Mozambique,MOZ,4.871829e+09,3.780891e+09,9.322958e+09,8.478513e+09,1.913268e+10,1.567264e+10,1.397695e+09,NaN,6.265720e+10
18,Canada,CAN,1.995863e+08,1.268382e+08,1.551723e+08,2.368324e+08,1.889218e+08,2.449042e+08,3.615595e+08,NaN,1.513815e+09
43,India,IND,1.175261e+08,9.415795e+07,1.080499e+08,1.334679e+08,1.331817e+08,1.558316e+08,1.595054e+08,NaN,9.017206e+08
91,South Africa,ZAF,1.180915e+08,8.083641e+07,1.212085e+08,9.455378e+07,1.476870e+08,9.058044e+07,1.091395e+08,NaN,7.620971e+08
3,Australia,AUS,7.336534e+07,1.079728e+08,9.376194e+07,9.721496e+07,1.306305e+08,8.665600e+07,9.456000e+07,NaN,6.841615e+08


In [10]:
# Столбчатая диаграмма
fig_bar = px.bar(
    pivot_import_volume.sort_values('TOTAL'),
    x='TOTAL',
    y='reporterDesc',
    orientation='h',
    title='ТОП-5 стран по объёму импорта (кг)',
    labels={'TOTAL': 'Экспорт (кг)', 'reporterDesc': 'Страна'}
)

# Карта (с лог-шкалой)
pivot_import_volume['log_TOTAL'] = np.log10(pivot_export_volume['TOTAL'] + 1)
fig_map = px.choropleth(
    pivot_import_volume,
    locations='reporterISO',
    color='log_TOTAL',
    hover_name='reporterDesc',
    labels={'log_TOTAL': 'log₁₀(кг)'},
    projection='natural earth'
)

# Комбинируем
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    specs=[[{"type": "choropleth"}, {"type": "bar"}]]
)

for trace in fig_map.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig_bar.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(title_text="Импорт", height=500)
fig.show()

### **6. Интерактивный виджет для среза по крупнейшим потребителям**

In [11]:
# Получаем уникальные годы и экспортеров
years = sorted(df_export['refYear'].dropna().unique())
exporters = sorted(df_export['reporterDesc'].dropna().unique())

# Виджеты
year_dropdown = widgets.Dropdown(options=years, description='Год:')
exporter_dropdown = widgets.Dropdown(options=exporters, description='Экспортёр:')

output = widgets.Output()

def update_table(change):
    with output:
        clear_output(wait=True)
        selected_year = year_dropdown.value
        selected_exporter = exporter_dropdown.value

        filtered = df_export[
            (df_export['refYear'] == selected_year) &
            (df_export['reporterDesc'] == selected_exporter)
        ][['partnerDesc', 'netWgt', 'primaryValue', 'Price per kg (USD)']].copy()

        # Сортировка по стоимости (убывание)
        filtered = filtered.sort_values('primaryValue', ascending=False)

        # Переименование колонок для удобства
        filtered.columns = ['Потребитель', 'Объём (кг)', 'Стоимость (USD)', 'Цена за кг (USD)']

        if filtered.empty:
            print("Нет данных для выбранных параметров.")
        else:
            # Округление
            filtered['Цена за кг (USD)'] = filtered['Цена за кг (USD)'].round(2)
            filtered['Стоимость (USD)'] = filtered['Стоимость (USD)'].round(0).astype(int)
            filtered['Объём (кг)'] = filtered['Объём (кг)'].round(0).astype(int)

            display(filtered.reset_index(drop=True))

# Привязка обновления
year_dropdown.observe(update_table, names='value')
exporter_dropdown.observe(update_table, names='value')

# Первичный вызов
update_table(None)

# Отображение виджетов и вывода
display(widgets.VBox([year_dropdown, exporter_dropdown, output]))

### **7. Cравнение по основным экспортерам и импортерам**

география поставок (объем по странам)
динамика цен от разных поставщиков для импортеров (или по разным потребителям для экспортеров)